# Generación de Contenido de Marketing con IA para ESU Analytics

Este notebook presenta el desarrollo final del proyecto de la materia *IA – Generación de Prompts*.  
El objetivo es mostrar cómo, mediante técnicas de **Fast Prompting**, es posible generar de manera eficiente publicaciones de marketing digital para **ESU Analytics**, consultora orientada a PyMEs de los sectores manufacturero y comercial.  

El proyecto aborda el problema de muchas PyMEs que no logran comunicar de forma clara y consistente el valor de la gestión de sus datos (ventas, precios, stock, finanzas).  
Para resolverlo, se implementa un flujo que combina:  

- **Texto → Texto**: generación de publicaciones en LinkedIn mediante el modelo `gpt-4o-mini`.  
- **Texto → Imagen**: creación de ilustraciones conceptuales con el modelo `gpt-image-1`.  
- **Fast Prompting** (zero-shot y one-shot), con control de tokens para optimizar costos.  
- Exportación automática en formatos `.json` y `.md`, asegurando trazabilidad y reproducibilidad.  

Este notebook integra todo el código y la documentación necesaria para replicar la solución final.

## Parámetros del brief

In [1]:
# Parámetros del brief que guían el prompt
brief_tema = "Cómo definir una estrategia comercial basada en datos"
brief_objetivo = "Generar interés en planificar ventas con indicadores claros"
brief_audiencia = "Gerentes de ventas y dueños de PyMEs industriales"
brief_insight = "Sin objetivos medibles, los equipos de ventas operan a ciegas"
brief_evidencia = "Empresas con tableros de indicadores mejoran la coordinación y reducen reprocesos"
brief_cta = "Conversemos sobre cómo diseñar tu tablero comercial"
brief_restricciones = "máx. 120 palabras, sin emojis ni tecnicismos excesivos"


# Ejemplo comentado de valores posibles:
# brief_tema = "Cómo ordenar datos de ventas para tomar decisiones mejor informadas",
# brief_objetivo = "Educar y generar interés por un diagnóstico inicial",
# brief_audiencia = "Dueños de PyMEs retail y gerentes comerciales",
# brief_insight = "Sin datos limpios y consistentes, el margen se vuelve impredecible",
# brief_evidencia = "Planillas dispersas generan errores; unificarlas reduce reprocesos",
# brief_cta = "Agendá una consulta gratuita para revisar tu caso",
# brief_restricciones = "máx. 110 palabras, sin hashtags ni emojis"

## Setup (variables y helpers)

In [2]:
# Importar librerías y configurar cliente
import os, time, json
from dotenv import load_dotenv
import tiktoken
from openai import OpenAI

In [3]:
# Cargar la API Key desde .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Tokenizer para modelos GPT-4o/mini
ENCODING_NAME = "o200k_base"
enc = tiktoken.get_encoding(ENCODING_NAME)

In [4]:
# Funciones auxiliares
def truncate_by_tokens(text: str, max_tokens: int) -> str:
    tokens = enc.encode(text)
    if len(tokens) <= max_tokens:
        return text
    return enc.decode(tokens[:max_tokens])

def count_tokens(text: str) -> int:
    return len(enc.encode(text))

def now_ms():
    return round(time.time() * 1000)

## Prompt builder (variables)

In [5]:
SYSTEM_PROMPT = """Eres un creador de contenido para ESU Analytics, consultora que ayuda a PyMEs de Manufactura y Comercio a profesionalizar su estrategia de datos (ventas, stock, estrategia comercial).
Tu prioridad: claridad, utilidad práctica para dueños/gerentes PyME en Argentina, tono profesional y cercano, y una llamada a la acción concreta.
Responde siempre en español de Buenos Aires neutral.
"""

USER_TEMPLATE = """Tarea: Escribe un post de LinkedIn.

Brief:
- Tema: {tema}
- Objetivo del post: {objetivo}
- Audiencia: {audiencia}
- Ángulo/Insight: {insight}
- Evidencia/ejemplo breve (opcional): {evidencia}
- CTA: {cta}
- Restricciones: {restricciones}

Formato deseado:
- 3–5 líneas, con una idea central y un cierre con CTA.
- Incluye 1 micro-lista de 3 ítems si aporta claridad.
- No uses hashtags en esta versión.

Entrega:
Devuelve SOLO un objeto JSON con:
{{
 "titulo": "...",
 "cuerpo": "...",
 "cta": "...",
 "palabras": <int>
}}
"""

def build_user_prompt(tema, objetivo, audiencia, insight, evidencia, cta, restricciones):
    return USER_TEMPLATE.format(
        tema=tema,
        objetivo=objetivo,
        audiencia=audiencia,
        insight=insight,
        evidencia=evidencia or "—",
        cta=cta,
        restricciones=restricciones
    )

## Llamada a OpenAI

In [6]:
def generate_post(brief, 
                  max_input_tokens=800, 
                  max_output_tokens=220, 
                  temperature=0.3):
    user_prompt = build_user_prompt(**brief)
    user_prompt = truncate_by_tokens(user_prompt, max_input_tokens)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_prompt}
    ]

    t0 = now_ms()
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=max_output_tokens,
        temperature=temperature
    )
    dt = now_ms() - t0

    text = resp.choices[0].message.content

    try:
        data = json.loads(text)
    except Exception:
        data = {"raw": text}

    meta = {
        "elapsed_ms": dt,
        "input_tokens_est": count_tokens(SYSTEM_PROMPT) + count_tokens(user_prompt),
        "output_tokens_est": count_tokens(text),
        "total_tokens_est": count_tokens(SYSTEM_PROMPT) + count_tokens(user_prompt) + count_tokens(text),
        "api": "chat.completions",
        "model": "gpt-4o-mini",
        "max_output_tokens": max_output_tokens,
        "temperature": temperature
    }
    # 👇 devolvemos también el prompt de usuario final para trazabilidad
    return data, meta, user_prompt

In [ ]:
# ============================
# Helpers de exportación
# ============================
import os, re, json, base64
from datetime import datetime

def ensure_export_dir(path="Exportado"):
    os.makedirs(path, exist_ok=True)
    return path

def next_post_id(path="Exportado"):
    """
    Escanea 'Exportado' y devuelve el próximo ID tipo '001', '002', ...
    Detecta archivos 'postXYZ.json' o 'postXYZ.md'
    """
    if not os.path.isdir(path):
        return "001"
    max_n = 0
    for fname in os.listdir(path):
        m = re.match(r"post(\d{3})\.(json|md)$", fname, re.IGNORECASE)
        if m:
            n = int(m.group(1))
            if n > max_n:
                max_n = n
    return f"{max_n+1:03d}"

def export_run(data, meta, brief, user_prompt, export_dir="Exportado"):
    """
    Guarda:
      - postXYZ.json con input, prompts, output y meta
      - postXYZ.md con texto listo para publicar
    """
    ensure_export_dir(export_dir)
    run_id = next_post_id(export_dir)
    ts = datetime.now().isoformat(timespec="seconds")

    # Paquete JSON completo (para trazabilidad)
    payload = {
        "timestamp": ts,
        "project": "ESU Analytics - Marketing con IA",
        "inputs": {
            "brief": brief,
            "system_prompt": SYSTEM_PROMPT,
            "user_prompt": user_prompt
        },
        "output": data,
        "meta": meta
    }

    json_path = os.path.join(export_dir, f"post{run_id}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

    # Markdown “listo para publicar”
    md_lines = []
    md_lines.append(f"# {data.get('titulo','(Sin título)')}")
    md_lines.append("")
    cuerpo = data.get("cuerpo") or data.get("raw") or ""
    md_lines.append(cuerpo)
    md_lines.append("")
    if "cta" in data and data["cta"]:
        md_lines.append(f"**CTA:** {data['cta']}")
        md_lines.append("")
    if "palabras" in data:
        md_lines.append(f"_Palabras: {data['palabras']}_")
        md_lines.append("")
    md_lines.append(f"<!-- post ID: {run_id} | {ts} | modelo: {meta.get('model')} | tokens aprox: {meta.get('total_tokens_est')} -->")

    md_path = os.path.join(export_dir, f"post{run_id}.md")
    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n".join(md_lines))

    return {"json": json_path, "md": md_path, "id": run_id}

# ============================
# Helpers para generar infografía con OpenAI Images
# ============================
def _extract_key_points_for_image(post, max_points=3):
    """
    Extrae 2–3 ideas clave del 'cuerpo' para orientar la infografía
    (sin texto literal; solo conceptos visuales).
    """
    cuerpo = (post.get("cuerpo") or post.get("raw") or "").strip()
    # tomar frases cortas
    parts = [p.strip(" -•*—–\n\r\t ") for p in re.split(r"[.\n]", cuerpo) if p.strip()]
    parts = [p for p in parts if len(p.split()) >= 3][:max_points]
    if not parts:
        parts = ["Orden de datos", "Decisiones informadas", "Control del negocio"]
    return parts[:max_points]

def build_image_prompt_from_post(post, brief):
    """
    Crea un prompt descriptivo para gpt-image-1,
    evitando pedir texto rendereado en la imagen.
    """
    titulo = post.get("titulo", "Contenido para PyMEs")
    puntos = _extract_key_points_for_image(post, max_points=3)
    cta = post.get("cta", "Agendá una consulta")


    prompt = f"""
    Ilustración estilo renderizado profesional, colores vibrantes y corporativos (azules, naranjas, verdes). Sin texto, números ni logotipos.
    - Idea central: {titulo}
    - Puntos clave: {", ".join(puntos)}
    Restricciones: sin personas, sin texto, sin números, sin logotipos.
    Sensación buscada: profesionalismo, transformación digital positiva.
    """



    return prompt.strip()

def generate_infographic_openai_image_from_post(
    client, post, brief, export_dir="Exportado", run_id="000",
    primary_size="1536x1024", fallback_size="1024x1024", low_cost=True
):
    """
    Genera una infografía con gpt-image-1 a partir del post generado.

    - Si low_cost=True → fuerza parámetros económicos (1024x1024, quality="low", n=1).
    - Si low_cost=False → intenta tamaño horizontal grande (ej. 1536x1024).
    - Si falla (403 u otro error) → fallback a tamaño menor o a Pillow si está disponible.
    """
    prompt_img = build_image_prompt_from_post(post, brief)

    def _gen_and_save(size, quality="low"):
        result = client.images.generate(
            model="gpt-image-1",
            prompt=prompt_img,
            size=size,
            quality=quality,
            n=1
        )
        b64 = result.data[0].b64_json
        img_bytes = base64.b64decode(b64)
        out_path = os.path.join(export_dir, f"post{run_id}_infografia_ai.png")
        with open(out_path, "wb") as f:
            f.write(img_bytes)
        return out_path

    try:
        if low_cost:
            # ✅ opción económica (aprox. USD 0.04 por imagen en 1024x1024)
            return _gen_and_save("1024x1024", quality="low")
        else:
            # ✅ opción premium horizontal
            return _gen_and_save(primary_size, quality="low")
    except Exception as e_primary:
        print(f"[Aviso] No se pudo generar en {primary_size if not low_cost else '1024x1024'}. Detalle: {e_primary}")
        try:
            return _gen_and_save(fallback_size)
        except Exception as e_fallback:
            print(f"[Aviso] Tampoco se pudo generar en {fallback_size}. Detalle: {e_fallback}")
            if PIL_OK:
                print("[Fallback] Generando infografía local con Pillow…")
                ensure_export_dir(export_dir)
                return render_infographic_pillow(post, brief, export_dir=export_dir, run_id=run_id)
            else:
                raise RuntimeError(
                    "No se pudo generar imagen con gpt-image-1 (acceso denegado) "
                    "y Pillow no está instalado. Verificá tu organización en OpenAI "
                    "o instalá Pillow con: pip install pillow"
                )



## Ejecución A/B

In [8]:
brief = {
    "tema": brief_tema,
    "objetivo": brief_objetivo,
    "audiencia": brief_audiencia,
    "insight": brief_insight,
    "evidencia": brief_evidencia,
    "cta": brief_cta,
    "restricciones": brief_restricciones
}

## Micro-evaluación

In [9]:
post, meta, user_prompt = generate_post(brief, temperature=0.2, max_output_tokens=220)
paths = export_run(post, meta, brief, user_prompt, export_dir="Exportado")

print("\n===== POST GENERADO =====")
print(json.dumps(post, ensure_ascii=False, indent=2))
print("\n===== MÉTRICAS =====")
print(meta)
print("\n===== ARCHIVOS GUARDADOS =====")
print(paths)


===== POST GENERADO =====
{
  "titulo": "Definiendo una Estrategia Comercial Basada en Datos",
  "cuerpo": "Sin objetivos medibles, los equipos de ventas operan a ciegas. Para optimizar tu estrategia comercial, considera estos tres puntos clave: 1) Establece indicadores claros, 2) Monitorea el rendimiento en tiempo real, 3) Ajusta tus tácticas según los datos. Empresas que implementan tableros de indicadores mejoran la coordinación y reducen reprocesos. No dejes que tu equipo navegue sin rumbo.",
  "cta": "Conversemos sobre cómo diseñar tu tablero comercial.",
  "palabras": 102
}

===== MÉTRICAS =====
{'elapsed_ms': 3479, 'input_tokens_est': 306, 'output_tokens_est': 143, 'total_tokens_est': 449, 'api': 'chat.completions', 'model': 'gpt-4o-mini', 'max_output_tokens': 220, 'temperature': 0.2}

===== ARCHIVOS GUARDADOS =====
{'json': 'Exportado\\post042.json', 'md': 'Exportado\\post042.md', 'id': '042'}


## Generación de imagen (OpenAI Images)

In [10]:
# Usamos el mismo ID que se generó al exportar el post para mantener trazabilidad
export_dir = "Exportado"
run_id = paths["id"]

# Generar infografía con OpenAI Images
img_path = generate_infographic_openai_image_from_post(
    client=client,
    post=post,
    brief=brief,
    export_dir=export_dir,
    run_id=run_id,
    low_cost=True  # Cambiar a False para intentar 1536x1024 (más caro)
)

print("\n===== Imagen =====")
print("Imagen guardada en:", img_path)



===== Imagen =====
Imagen guardada en: Exportado\post042_infografia_ai.png


# Conclusiones

El proyecto logró cumplir con los objetivos propuestos:  
- Se resolvió la problemática de generar contenido claro y atractivo para PyMEs, utilizando exclusivamente modelos de IA generativa.  
- Se aplicaron técnicas de **Fast Prompting** para obtener resultados consistentes y reducir costos mediante control de tokens.  
- Se integraron de manera efectiva dos modalidades: **texto–texto** y **texto–imagen**, cumpliendo con los requisitos del curso.  
- Se garantizó la **trazabilidad** de cada ejecución, almacenando inputs, outputs y métricas de uso.  

Los resultados muestran que la metodología es **viable, escalable y orientada a resultados**, aportando valor tanto en el plano académico (demostración de conceptos aprendidos) como en el profesional (aplicación real en la estrategia digital de ESU Analytics).  

**Próximos pasos sugeridos:** ampliar la variedad de briefs, experimentar con estilos gráficos alternativos y medir métricas reales de impacto en redes sociales (engagement, alcance).  